In [1]:
import yaml
import pandas as pd
import numpy as np
from IPython.display import display

This file requires three inputs: result and ground truth csv files as well as the prompt file. Samples of all files are included inside 'data' folder.

This file:
1. Generates comparisons of result and ground truth values and calculates accuracy paper-wise (row-wise) and sub-criteria-wise (column wise).
2. Calculates two tables based on per-sub-criteria values: per-paper metrics and per-sub-criteria metrics.
3. Calculates risk-level per criteria.
4. Generates comparisons of result and ground truth risk-level per criteria values and calculates accuracy paper-wise (row-wise) and risk-level per criteria-wise (column wise).
5. Calculates metrics for risk-level per-criteria (confusion matrix).
6. Save all results (separated into multiple sheets) into an excel file.

Make sure the paper_id (paper title) column is the same and alphabetically increasing ordered for both ground truth and LLM results. The order and number of the criteria columns need to be similar aswell for both ground truth and LLM results.

# Configuration

In [2]:
# =========================
# Input files.
# =========================
LLM_CSV_PATH = "data/llm_train.csv"     # LLM results
GT_CSV_PATH  = "data/lsr_train.csv"    # ground truth
PROMPT_PATH = "data/prompt.yaml" # prompt file.

In [3]:
# =========================
# Config Continued
# =========================

# Column in BOTH CSVs that contains the paper identifier (first column by position).
PAPER_ID_COL = "paper_id"

# Predefined SUB_CRITERIA.
# SUB_CRITERIA = [
#     "criteria 1.1", "criteria 1.2", "criteria 1.3", "criteria 1.4", "criteria 1.5",
#     "criteria 2.1", "criteria 2.3", "criteria 2.4", "criteria 2.5", "criteria 2.6",
#     "criteria 5.1",
#     "criteria 6.1", "criteria 6.2", "criteria 6.3",
#     "criteria 7.1", "criteria 7.2", "criteria 7.3", "criteria 7.4",
# ]

# Generated SUB_CRITERIA from prompt.yaml.
with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    doc = yaml.safe_load(f) or {}

nested_subs = {
    crit["id"]: {
        sub["id"]: {
            "title": sub.get("title", ""),
            "instruction": sub.get("instruction", "")
        }
        for sub in crit.get("sub_criteria", [])
    }
    for crit in doc.get("Criteria", [])
}

SUB_CRITERIA = []
for criteria_id, sub_crit_dict in nested_subs.items():
    for sub_crit_id, sub_crit in sub_crit_dict.items():
        SUB_CRITERIA.append(f"criteria {sub_crit_id}")

# Presentation
DISPLAY_SEPARATOR = " / "  # string between LLM and GT values in a cell.

This script will take the PAPER_IDS and CRITERIA from either one dataframe (either ground truth or LLM results) and apply it to the other dataframe. 
Hence, naming isn't important here, but ordering of the paper and criteria will be important. Ordering has to be the same between both dataset.

# Result Comparison

In [4]:
# =========================
# Imports & helpers
# =========================

def _canon_label(x: object) -> str:
    """Converts all into lowercase; map 'probably yes/no' to 'yes/no'; empty for NaN."""
    if pd.isna(x):
        return ""
    s = str(x).strip().lower()
    if s == "probably yes":
        s = "yes"
    elif s == "probably no":
        s = "no"
    return s

def _clean_col(s: str) -> str:
    """Simple normalization for column names (used in string mode)."""
    return " ".join(str(s).strip().lower().split())

def load_by_position(dataframe: pd.DataFrame, paper_ids: list[str], criteria: list[str], paper_id_col: str) -> pd.DataFrame:
    """
    Read CSV assuming:
    - First column (by position) is the paper id, and rows appear in the same order as PAPER_IDS
    - Next len(criteria) columns (by position) correspond to CRITERIA in that exact order
    Any extra columns after criteria are ignored. Column *names* in the file don't matter.
    """
    df = dataframe
    need_cols = 1 + len(criteria)
    if df.shape[1] < need_cols:
        raise ValueError(f"Dataframe: expected at least {need_cols} columns (id + {len(criteria)} criteria), got {df.shape[1]}.")

    df = df.iloc[:, :need_cols]

    # Rename columns (first col -> PAPER_ID_COL, rest -> CRITERIA).
    df.columns = [paper_id_col] + criteria

    # Enforce row count/order by position
    if df.shape[0] != len(paper_ids):
        raise ValueError(f"Dataframe: expected exactly {len(paper_ids)} rows (one per paper), got {df.shape[0]}.")

    df[paper_id_col] = paper_ids
    df = df.set_index(paper_id_col)

    # Canonicalize labels
    for c in criteria:
        df[c] = df[c].apply(_canon_label)

    return df

def build_comparison(llm_df: pd.DataFrame, gt_df: pd.DataFrame, paper_ids: list[str], criteria: list[str], sep: str) -> pd.DataFrame:
    """Return a criteria × papers table with 'llm / ground truth' strings."""
    out = pd.DataFrame(index=criteria, columns=paper_ids, dtype=object)
    for p in paper_ids:
        for c in criteria:
            pred  = "" if pd.isna(llm_df.loc[p, c]) else str(llm_df.loc[p, c])
            truth = "" if pd.isna(gt_df.loc[p, c])  else str(gt_df.loc[p, c])
            out.at[c, p] = f"{pred}{sep}{truth}"
            
    # Overall Accuracy:
    acc_per_paper_list = []
    for p in paper_ids:
        for c in criteria:
            acc_per_paper_list.append(str(llm_df.loc[p, c]) == str(gt_df.loc[p, c]))
            
    return out, np.mean(acc_per_paper_list).round(3)

In [5]:
# =========================
# Import Data & Wrangle
# =========================
df_llm_raw = pd.read_csv(LLM_CSV_PATH)
df_gt_raw = pd.read_csv(GT_CSV_PATH)

In [6]:
# Unformatted LLM results.
df_llm_raw.head()

,no,file_name,1.1) Confounder Possibility,1.2) Confounder All,1.3) Confounder Justified Omission,1.4) Confounder Accuracy,1.5) Confounder Analysis,2.1) SampleExchangeability,2.3) Sample Exclusion,2.4) SampleGroupComparability,2.5) Sample Group Difference Intervention,2.6) Sample Bias Adjustment,5.1) Aware of Study,6.1) DataSelective,6.2) Data_Subgroups,6.3) Data_Causal,7.1) StatsRecording,7.2) Stats_Descriptive_Error,7.3) Stats_Inferential_Error,7.4) Stats_Inferential_Violation
0,1,"Allcott, H (2011).md",no,yes,no,no,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,no,yes,no
1,2,"Ayres, Raseman, Shih, 2012.md",yes,yes,no,yes,yes,yes,yes,no,yes,yes,yes,yes,no,yes,yes,yes,no,no
2,3,"Bager, S; Mundaca, L (2017).md",yes,no,no,yes,yes,yes,yes,yes,yes,yes,yes,yes,no,yes,yes,yes,yes,yes
3,4,"Carroll, J; Lyons, S; Denny, E (2014).md",yes,no,no,yes,yes,yes,yes,yes,yes,yes,yes,yes,no,yes,yes,yes,yes,no
4,5,"Houde, S; Todd, A; Sudarshan, A; Flora, JA; Ar...",yes,yes,no,no,yes,yes,yes,yes,yes,yes,yes,yes,no,yes,yes,yes,yes,no


In [7]:
# Unformatted ground truth results.
df_gt_raw.head()

,effectID,coder,Authors,documentType,document PY,document title,data_type,abstract,StudyYear,ConfounderPossibility,...,SampleGroupDifferenceIntervention,SampleBiasAdjustment,AwareofStudy,DataSelective,DataSubgroups,DataCausal,StatsRecording,StatsDescriptiveError,StatsInferentialError,StatsInferentialViolation
0,23-208-1,NaN,"Allcott, H",NaN,2011,Social norms and energy conservation,training,This paper evaluates a series of programs run ...,2011,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No
1,23-215-1,NaN,"Ayres, I; Raseman, S; Shih, A",NaN,2013,Evidence from Two Large Field Experiments that...,training,By providing feedback to customers on home ele...,2013,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No
2,23-216-1,NaN,"Bager, S; Mundaca, L",NaN,2017,Making 'Smart Meters' smarter? Insights from a...,training,This paper examines the relationship between l...,2017,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No
3,23-225-1,NaN,"Carroll, J; Lyons, S; Denny, E",NaN,2014,Reducing household electricity demand through ...,training,The international roll out of residential smar...,2014,Yes,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,No
4,23-257-1,NaN,"Houde, S; Todd, A; Sudarshan, A; Flora, JA; Ar...",NaN,2013,Real-time Feedback and Electricity Consumption...,training,Real-time information feedback delivered via t...,2013,Yes,...,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,No


In [8]:
# Wrangle ground truth dataframe, drops unnecessary columns, standarizes column names.
df_gt = df_gt_raw.drop(columns=['effectID', 'coder', 'documentType', 'document PY', 'document title', 'data_type', 'abstract', 'StudyYear'])
df_gt.columns = [PAPER_ID_COL] + SUB_CRITERIA
df_gt = df_gt.applymap(_canon_label)
df_gt.head()

,paper_id,criteria 1.1,criteria 1.2,criteria 1.3,criteria 1.4,criteria 1.5,criteria 2.1,criteria 2.3,criteria 2.4,criteria 2.5,criteria 2.6,criteria 5.1,criteria 6.1,criteria 6.2,criteria 6.3,criteria 7.1,criteria 7.2,criteria 7.3,criteria 7.4
0,"allcott, h",yes,no,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,no,no,no
1,"ayres, i; raseman, s; shih, a",yes,no,no,yes,yes,yes,yes,no,yes,yes,yes,yes,yes,yes,yes,no,no,no
2,"bager, s; mundaca, l",yes,no,no,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,no
3,"carroll, j; lyons, s; denny, e",yes,no,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,no,no,no
4,"houde, s; todd, a; sudarshan, a; flora, ja; ar...",yes,no,no,yes,yes,yes,yes,yes,yes,yes,no,yes,yes,yes,yes,yes,yes,no


In [9]:
# Take PAPER_ID_COL from one of the dataframe.
PAPER_IDS = df_gt[PAPER_ID_COL].astype(str).tolist()
PAPER_IDS

['allcott, h',
 'ayres, i; raseman, s; shih, a',
 'bager, s; mundaca, l',
 'carroll, j; lyons, s; denny, e',
 'houde, s; todd, a; sudarshan, a; flora, ja; armel, kc',
 'matsukawa, i.']

In [10]:
# Apply PAPER_ID_COL from other dataframe to the other dataframe (standarizing).
df_llm = df_llm_raw.drop(columns=['no'])
df_llm.columns = ["paper_id"] + SUB_CRITERIA
df_llm["paper_id"] = PAPER_IDS
df_llm = df_llm.applymap(_canon_label)
df_llm.head()

,paper_id,criteria 1.1,criteria 1.2,criteria 1.3,criteria 1.4,criteria 1.5,criteria 2.1,criteria 2.3,criteria 2.4,criteria 2.5,criteria 2.6,criteria 5.1,criteria 6.1,criteria 6.2,criteria 6.3,criteria 7.1,criteria 7.2,criteria 7.3,criteria 7.4
0,"allcott, h",no,yes,no,no,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,yes,no,yes,no
1,"ayres, i; raseman, s; shih, a",yes,yes,no,yes,yes,yes,yes,no,yes,yes,yes,yes,no,yes,yes,yes,no,no
2,"bager, s; mundaca, l",yes,no,no,yes,yes,yes,yes,yes,yes,yes,yes,yes,no,yes,yes,yes,yes,yes
3,"carroll, j; lyons, s; denny, e",yes,no,no,yes,yes,yes,yes,yes,yes,yes,yes,yes,no,yes,yes,yes,yes,no
4,"houde, s; todd, a; sudarshan, a; flora, ja; ar...",yes,yes,no,no,yes,yes,yes,yes,yes,yes,yes,yes,no,yes,yes,yes,yes,no


In [11]:
llm = load_by_position(df_llm, PAPER_IDS, SUB_CRITERIA, PAPER_ID_COL)
gt  = load_by_position(df_gt,  PAPER_IDS, SUB_CRITERIA, PAPER_ID_COL)

# =========================
# Build and display comparison
# =========================
comparison, acc = build_comparison(llm, gt, PAPER_IDS, SUB_CRITERIA, DISPLAY_SEPARATOR)

# Add accuracy column per-criterion (row)
# Definition: fraction of papers where LLM == GT for that criterion.
# Denominator uses papers with non-empty ground-truth for that criterion.
acc_values = []
for crit in SUB_CRITERIA:
    preds = llm.loc[PAPER_IDS, crit]
    truths = gt.loc[PAPER_IDS, crit]
    mask = truths.astype(str).str.len() > 0  # count only where GT present
    total = int(mask.sum())
    matches = int((preds[mask] == truths[mask]).sum()) if total > 0 else 0
    acc_pct = (matches / total * 100.0) if total > 0 else None
    formatted = f"{matches}/{total} ({acc_pct:.1f}%)" if total > 0 else "n/a"
    acc_values.append(formatted)
    
comparison["accuracy_subcriteria_wise"] = acc_values
    
# Add paper-wise accuracy at the bottom row (per column / paper)
# Definition: fraction of criteria where LLM == GT for that paper
acc_bottom = {}
for p in PAPER_IDS:
    preds = llm.loc[p, SUB_CRITERIA]
    truths = gt.loc[p, SUB_CRITERIA]
    total = len(SUB_CRITERIA)
    matches = int((preds.values == truths.values).sum())
    acc_pct = matches / total * 100.0
    acc_bottom[p] = f"{matches}/{total} ({acc_pct:.1f}%)"

# Append as the last row
comparison.loc["accuracy_paper wise"] = pd.Series(acc_bottom)

direct_comparison_df = comparison.T
direct_comparison_df.iat[-1,-1] = f"overall: {acc}"

direct_comparison_df = direct_comparison_df.reset_index().rename(columns={"index": PAPER_ID_COL})

print("=== Criteria x Papers (LLM / Ground Truth) with per-criterion accuracy ===")
display(direct_comparison_df)

=== Criteria x Papers (LLM / Ground Truth) with per-criterion accuracy ===


,paper_id,criteria 1.1,criteria 1.2,criteria 1.3,criteria 1.4,criteria 1.5,criteria 2.1,criteria 2.3,criteria 2.4,criteria 2.5,criteria 2.6,criteria 5.1,criteria 6.1,criteria 6.2,criteria 6.3,criteria 7.1,criteria 7.2,criteria 7.3,criteria 7.4,accuracy_paper wise
0,"allcott, h",no / yes,yes / no,no / yes,no / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,no / no,yes / no,no / no,13/18 (72.2%)
1,"ayres, i; raseman, s; shih, a",yes / yes,yes / no,no / no,yes / yes,yes / yes,yes / yes,yes / yes,no / no,yes / yes,yes / yes,yes / yes,yes / yes,no / yes,yes / yes,yes / yes,yes / no,no / no,no / no,15/18 (83.3%)
2,"bager, s; mundaca, l",yes / yes,no / no,no / no,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,no / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / no,16/18 (88.9%)
3,"carroll, j; lyons, s; denny, e",yes / yes,no / no,no / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,no / yes,yes / yes,yes / yes,yes / no,yes / no,no / no,14/18 (77.8%)
4,"houde, s; todd, a; sudarshan, a; flora, ja; ar...",yes / yes,yes / no,no / no,no / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / no,yes / yes,no / yes,yes / yes,yes / yes,yes / yes,yes / yes,no / no,14/18 (77.8%)
5,"matsukawa, i.",yes / yes,no / no,no / yes,yes / yes,yes / yes,yes / yes,yes / yes,yes / no,yes / yes,yes / no,yes / yes,yes / yes,yes / yes,yes / no,yes / yes,yes / yes,yes / no,no / yes,12/18 (66.7%)
6,accuracy_subcriteria_wise,5/6 (83.3%),3/6 (50.0%),3/6 (50.0%),4/6 (66.7%),6/6 (100.0%),6/6 (100.0%),6/6 (100.0%),5/6 (83.3%),6/6 (100.0%),5/6 (83.3%),5/6 (83.3%),6/6 (100.0%),2/6 (33.3%),5/6 (83.3%),6/6 (100.0%),4/6 (66.7%),3/6 (50.0%),4/6 (66.7%),overall: 0.778


# Per-Criteria Wise Metrics

In [12]:
from collections import defaultdict

def compute_per_paper_metrics_table(df_ref, df_res, filename):
    df_ref_lower = df_ref.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    df_res_lower = df_res.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    criteria_cols = [col for col in df_ref.columns if col not in ['paper_id']]

    # set positive as high.
    positive = "yes"
    negative = "no"

    records = []
    for col in criteria_cols:
        for i, (ref_val, res_val) in enumerate(zip(df_ref_lower[col], df_res_lower[col])):
            case_id = df_ref.loc[i, "paper_id"]

            if ref_val == positive:
                if res_val == positive:
                    classification = "TP"
                elif res_val == negative:
                    classification = "FN"
                else:
                    classification = "Unclear"
            elif ref_val == negative:
                if res_val == positive:
                    classification = "FP"
                elif res_val == negative:
                    classification = "TN"
                else:
                    classification = "Unclear"
            else:
                classification = "Unclear"

            records.append({
                "ID": case_id,
                "Criterion": col,
                "Reference": ref_val,
                "Prediction": res_val,
                "Classification": classification
            })

    comparison_df = pd.DataFrame(records)

    study_stats = defaultdict(lambda: {"TP": 0, "TN": 0, "FP": 0, "FN": 0})

    for _, row in comparison_df.iterrows():
        study_id = row["ID"]
        cls = row["Classification"]
        if cls in study_stats[study_id]:
            study_stats[study_id][cls] += 1

    study_metrics = []

    for study_id, counts in study_stats.items():
        TP = counts["TP"]
        TN = counts["TN"]
        FP = counts["FP"]
        FN = counts["FN"]
        total = TP + TN + FP + FN

        accuracy = (TP + TN) / total if total > 0 else 0
        sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        f1_score = 2 * (precision * sensitivity) / (precision + sensitivity) if (precision + sensitivity) > 0 else 0

        study_metrics.append({
            "ID": study_id,
            "TP": TP,
            "TN": TN,
            "FP": FP,
            "FN": FN,
            "Correct assessment rate": round(accuracy, 4),
            "Sensitivity": round(sensitivity, 4),
            "Specificity": round(specificity, 4),
            "Precision": round(precision, 4),
            "F-score": round(f1_score, 4),
        })

    study_metrics_df = pd.DataFrame(study_metrics)
    return study_metrics_df
    
def compute_per_subcriteria_metrics_table(df_ref, df_res1):
    df_ref = df_ref.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    df_res1 = df_res1.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    criteria_cols = [col for col in df_ref.columns if col not in ['paper_id']]
    positive = "yes"
    negative = "no"

    def classify_entries(df_result):
        records = []
        for col in criteria_cols:
            for i, (ref_val, res_val) in enumerate(zip(df_ref[col], df_result[col])):
                #case_id = df_ref.loc[i, "paper_id"]
                case_id = df_ref.iloc[i, df_ref.columns.get_loc("paper_id")]

                if ref_val == positive:
                    if res_val == positive:
                        cls = "TP"
                    elif res_val == negative:
                        cls = "FN"
                    else:
                        cls = "Unclear"
                elif ref_val == negative:
                    if res_val == positive:
                        cls = "FP"
                    elif res_val == negative:
                        cls = "TN"
                    else:
                        cls = "Unclear"
                else:
                    cls = "Unclear"

                records.append({
                    "ID": case_id,
                    "Criterion": col,
                    "Reference": ref_val,
                    "Prediction": res_val,
                    "Classification": cls
                })
        return pd.DataFrame(records)

    comp_df1 = classify_entries(df_res1)

    def compute_domain_metrics(comp_df):
        domain_stats = defaultdict(lambda: {"TP": 0, "TN": 0, "FP": 0, "FN": 0})
        for _, row in comp_df.iterrows():
            domain = row["Criterion"]
            cls = row["Classification"]
            if cls in domain_stats[domain]:
                domain_stats[domain][cls] += 1

        domain_metrics = []
        for domain, counts in domain_stats.items():
            TP = counts["TP"]
            TN = counts["TN"]
            FP = counts["FP"]
            FN = counts["FN"]
            total = TP + TN + FP + FN

            acc = (TP + TN) / total if total > 0 else 0
            sens = TP / (TP + FN) if (TP + FN) > 0 else None
            spec = TN / (TN + FP) if (TN + FP) > 0 else None
            prec = TP / (TP + FP) if (TP + FP) > 0 else None
            f1 = (
                2 * (prec * sens) / (prec + sens)
                if prec is not None and sens is not None and (prec + sens) > 0
                else None
            )

            domain_metrics.append({
                "Domain": domain,
                "TP": TP, "TN": TN, "FP": FP, "FN": FN,
                "Accuracy": round(acc, 4),
                "Sensitivity": round(sens, 4) if sens is not None else "Not available",
                "Specificity": round(spec, 4) if spec is not None else "Not available",
                "Precision": round(prec, 4) if prec is not None else "Not available",
                "F-score": round(f1, 4) if f1 is not None else "Not available",
            })
        return pd.DataFrame(domain_metrics)

    df1 = compute_domain_metrics(comp_df1)

    avg_rows = []
    for _, row in df1.iterrows():
        avg_rows.append({
            "Domain": row["Domain"],
            "TP": row["TP"],
            "TN": row["TN"],
            "FP": row["FP"],
            "FN": row["FN"],
            "Correct assessment rate": row["Accuracy"],
            "Sensitivity": row["Sensitivity"],
            "Specificity": row["Specificity"],
            "Precision": row["Precision"],
            "F-score": row["F-score"]})

    avg_df = pd.DataFrame(avg_rows)
    return avg_df

In [13]:
per_paper_metrics_df = compute_per_paper_metrics_table(df_gt, df_llm, "result_test")
per_paper_metrics_df.head()

,ID,TP,TN,FP,FN,Correct assessment rate,Sensitivity,Specificity,Precision,F-score
0,"allcott, h",11,2,2,3,0.7222,0.7857,0.5000,0.8462,0.8148
1,"ayres, i; raseman, s; shih, a",11,4,2,1,0.8333,0.9167,0.6667,0.8462,0.8800
2,"bager, s; mundaca, l",14,2,1,1,0.8889,0.9333,0.6667,0.9333,0.9333
3,"carroll, j; lyons, s; denny, e",12,2,2,2,0.7778,0.8571,0.5000,0.8571,0.8571
4,"houde, s; todd, a; sudarshan, a; flora, ja; ar...",12,2,2,2,0.7778,0.8571,0.5000,0.8571,0.8571


In [14]:
per_subcriteria_metrics_df = compute_per_subcriteria_metrics_table(df_gt, df_llm)
per_subcriteria_metrics_df

,Domain,TP,TN,FP,FN,Correct assessment rate,Sensitivity,Specificity,Precision,F-score
0,criteria 1.1,5,0,0,1,0.8333,0.8333,Not available,1.0,0.9091
1,criteria 1.2,0,3,3,0,0.5000,Not available,0.5,0.0,Not available
2,criteria 1.3,0,3,0,3,0.5000,0.0,1.0,Not available,Not available
3,criteria 1.4,4,0,0,2,0.6667,0.6667,Not available,1.0,0.8
4,criteria 1.5,6,0,0,0,1.0000,1.0,Not available,1.0,1.0
5,criteria 2.1,6,0,0,0,1.0000,1.0,Not available,1.0,1.0
6,criteria 2.3,6,0,0,0,1.0000,1.0,Not available,1.0,1.0
7,criteria 2.4,4,1,1,0,0.8333,1.0,0.5,0.8,0.8889
8,criteria 2.5,6,0,0,0,1.0000,1.0,Not available,1.0,1.0
9,criteria 2.6,5,0,1,0,0.8333,1.0,0.0,0.8333,0.9091


# Per-Criteria Risk-Level Calculation
Unlike direct per-criteria calculation, this script is specific to the prompt and criteria structure that we have.

In [15]:
# =========================
# Risk calculation
# =========================

import pandas as pd
from typing import Callable, Dict, List, Tuple

_RISK_TO_NUM = {"low": 0, "med": 1, "high": 2}
_NUM_TO_RISK = {v: k for k, v in _RISK_TO_NUM.items()}


def _yn(x) -> str:
    """Normalize to yes/no (datasets should already be formatted)."""
    #if pd.isna(x): return "no"
    s = str(x).strip().lower()
    if s in {"yes", "y", "true", "1"}: return "yes"
    if s in {"no", "n", "false", "0"}: return "no"
    return "yes" if "yes" in s else "no"

# =========================
# Rules from flowcharts found in CEE handbook.
# =========================

RULES_CRIT_1: Dict[Tuple[str, str, str, str, str], str] = {
    # All decisions starting with 1.1 = no is coded in the risk_from_flow_1()
    ('yes', 'yes', 'yes', 'yes', 'yes'): 'low',
    ('yes', 'yes', 'no', 'yes', 'yes'): 'low',
    
    ('yes', 'yes', 'yes', 'no', 'yes'): 'low',
    ('yes', 'yes', 'no', 'no', 'yes'): 'low',
    
    ('yes', 'yes', 'yes', 'no', 'no'): 'med',
    ('yes', 'yes', 'no', 'no', 'no'): 'med',
    ('yes', 'yes', 'no', 'yes', 'no'): 'med',
    
    ('yes', 'no', 'yes', 'yes', 'yes'): 'low',
    ('yes', 'no', 'yes', 'no', 'yes'): 'low',
    ('yes', 'no', 'yes', 'no', 'no'): 'high',
    
    ('yes', 'no', 'no', 'no', 'no'): 'high',
    ('yes', 'no', 'no', 'yes', 'no'): 'high',
    ('yes', 'no', 'no', 'no', 'yes'): 'high',
    ('yes', 'no', 'no', 'yes', 'yes'): 'high',
}

def risk_from_flow_1(row: pd.Series,
                     cols: List[str] = ["criteria 1.1", "criteria 1.2", "criteria 1.3", "criteria 1.4", "criteria 1.5"],
                     rules: Dict[Tuple[str, str, str, str, str], str] = RULES_CRIT_1,
                     default: str = "NA") -> str:
    if row['criteria 1.1'] == 'no':
        return 'low'
    else:
        values = tuple(_yn(row[c]) for c in cols)
        return rules.get(values, default)

RULES_CRIT_2: Dict[Tuple[str, str, str, str, str], str] = {
    ('yes', 'no', 'no', 'no', 'no'): 'low',
    ('yes', 'no', 'yes', 'no', 'no'): 'low',
    ('yes', 'no', 'no', 'yes', 'no'): 'low',
    ('yes', 'no', 'no', 'no', 'yes'): 'low',
    ('yes', 'no', 'yes', 'yes', 'no'): 'low',
    ('yes', 'no', 'no', 'yes', 'yes'): 'low',
    ('yes', 'no', 'yes', 'no', 'yes'): 'low',
    ('yes', 'no', 'yes', 'yes', 'yes'): 'low',
    
    ('yes', 'yes', 'yes', 'no', 'no'): 'med',
    ('yes', 'yes', 'yes', 'yes', 'no'): 'med',
    ('yes', 'yes', 'yes', 'no', 'yes'): 'med',
    ('yes', 'yes', 'yes', 'yes', 'yes'): 'med',
    
    ('yes', 'yes', 'no', 'no', 'no'): 'med',
    ('yes', 'yes', 'no', 'no', 'yes'): 'med',
    ('yes', 'yes', 'yes', 'no', 'yes'): 'med',
    
    ('yes', 'yes', 'no', 'yes', 'no'): 'high',
    ('yes', 'yes', 'no', 'yes', 'yes'): 'med',
    
    ('no', 'no', 'yes', 'no', 'no'): 'med',
    ('no', 'no', 'yes', 'yes', 'no'): 'med',
    ('no', 'no', 'yes', 'no', 'yes'): 'med',
    ('no', 'no', 'yes', 'yes', 'yes'): 'med',
    
    ('no', 'no', 'no', 'no', 'no'): 'med',
    ('no', 'no', 'no', 'no', 'yes'): 'med',
    
    ('no', 'no', 'no', 'yes', 'no'): 'high',
    ('no', 'no', 'no', 'yes', 'yes'): 'med',
    
    
    ('no', 'yes', 'yes', 'no', 'no'): 'med',
    ('no', 'yes', 'yes', 'yes', 'no'): 'med',
    ('no', 'yes', 'yes', 'no', 'yes'): 'med',
    ('no', 'yes', 'yes', 'yes', 'yes'): 'med',
    
    ('no', 'yes', 'no', 'no', 'no'): 'med',
    ('no', 'yes', 'no', 'no', 'yes'): 'med',
    
    ('no', 'yes', 'no', 'yes', 'no'): 'high',
    ('no', 'yes', 'no', 'yes', 'yes'): 'med',
}

def risk_from_flow_2(row: pd.Series,
                     cols: List[str] = ["criteria 2.1", "criteria 2.3", "criteria 2.4", "criteria 2.5", "criteria 2.6"],
                     rules: Dict[Tuple[str, str, str, str, str], str] = RULES_CRIT_2,
                     default: str = "NA") -> str:
    values = tuple(_yn(row[c]) for c in cols)
    return rules.get(values, default)

RULES_CRIT_5: Dict[Tuple[str], str] = {} # stub.

def risk_from_flow_5(row: pd.Series,
                     cols: List[str] = ["criteria 5.1"],
                     rules: Dict[Tuple[str], str] = RULES_CRIT_5,
                     default: str = "NA") -> str:
    
    if row['criteria 5.1'] == 'no':
        return 'low'
    else:
        return 'high'

RULES_CRIT_6: Dict[Tuple[str, str, str], str] = {
    ("no", "no", "no"): "low",
    ("no", "no", "yes"):  "med",
    ("no", "yes", "no"): "med",
    ("no", "yes", "yes"): "high",
    ("yes", "no",  "no"):  "med",
    ("yes",  "no", "yes"): "high",
    ("yes",  "yes", "no"):  "high",
    ("yes",  "yes",  "yes"): "high",
}

def risk_from_flow_6(row: pd.Series,
                     cols: List[str] = ["criteria 6.1", "criteria 6.2", "criteria 6.3"],
                     rules: Dict[Tuple[str, str, str], str] = RULES_CRIT_6,
                     default: str = "NA") -> str:
    values = tuple(_yn(row[c]) for c in cols)
    return rules.get(values, default)

RULES_CRIT_7: Dict[Tuple[str, str, str, str], str] = {
    ('no', 'no', 'no', 'no'): 'low',
    ('no', 'no', 'no', 'yes'): 'low', #
    ('no', 'no', 'yes', 'no'): 'high',
    ('no', 'no', 'yes', 'yes'): 'high', #
    ('no', 'yes', 'no', 'no'): 'high',
    ('no', 'yes', 'yes', 'no'): 'high',
    ('no', 'yes', 'no', 'yes'): 'high',
    ('no', 'yes', 'yes', 'yes'): 'high', #
    ('yes', 'no', 'no', 'no'): 'med',
    ('yes', 'no', 'no', 'yes'): 'med', #
    ('yes', 'no', 'yes', 'no'): 'high',
    ('yes', 'no', 'yes', 'yes'): 'high', #
    ('yes', 'yes', 'no', 'no'): 'high',
    ('yes', 'yes', 'yes', 'no'): 'high',
    ('yes', 'yes', 'no', 'yes'): 'high',
    ('yes', 'yes', 'yes', 'yes'): 'high',
}

def risk_from_flow_7(row: pd.Series,
                     cols: List[str] = ["criteria 7.1", "criteria 7.2", "criteria 7.3", "criteria 7.4"],
                     rules: Dict[Tuple[str, str, str, str], str] = RULES_CRIT_7,
                     default: str = "NA") -> str:
    values = tuple(_yn(row[c]) for c in cols)
    return rules.get(values, default)

# =========================
# Define top-level criteria.
# Maps criterion name -> function(row)-> 'low'/'med'/'high'.
# =========================

CRITERION_FUNCS: Dict[str, Callable[[pd.Series], str]] = {
    "criterion_1": risk_from_flow_1,
    "criterion_2": risk_from_flow_2,
    "criterion_5": risk_from_flow_5,
    "criterion_6": risk_from_flow_6,
    "criterion_7": risk_from_flow_7,
}

# =========================
# Main APIs
# =========================
def calculate_risk_paper(row: pd.Series,
                         paper_title_col: str = PAPER_ID_COL,
                         crit_funcs: Dict[str, Callable[[pd.Series], str]] = CRITERION_FUNCS
                         ) -> Tuple[str, Dict[str, str], str]:
    """
    Returns:
      paper_title,
      risk_level_per_criteria: dict {criterion_name: 'low'|'med'|'high'},
    """
    per_crit = {crit_name: fn(row) for crit_name, fn in crit_funcs.items()}
    return str(row[paper_title_col]), per_crit

def calculate_risk_all_papers(df: pd.DataFrame,
                              paper_title_col: str = PAPER_ID_COL,
                              crit_funcs: Dict[str, Callable[[pd.Series], str]] = CRITERION_FUNCS
                              ) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Applies calculate_risk_paper() over all rows.
    Returns:
      summary_df: columns ['paper_title','risk_level']
      details_df: columns ['paper_title', <one per criterion>, 'overall']
    """
    titles: List[str] = []
    percrit_rows: List[Dict[str, str]] = []

    for _, row in df.iterrows():
        title, percrit = calculate_risk_paper(row, paper_title_col, crit_funcs)
        titles.append(title)
        percrit_rows.append(percrit)

    details_df = pd.DataFrame(percrit_rows)
    details_df.insert(0, PAPER_ID_COL, titles)

    return details_df


In [16]:
llm_risk_df = calculate_risk_all_papers(df_llm, paper_title_col=PAPER_ID_COL, crit_funcs=CRITERION_FUNCS)
display(llm_risk_df)

gt_risk_df = calculate_risk_all_papers(df_gt, paper_title_col=PAPER_ID_COL, crit_funcs=CRITERION_FUNCS)
display(gt_risk_df)

,paper_id,criterion_1,criterion_2,criterion_5,criterion_6,criterion_7
0,"allcott, h",low,med,high,high,high
1,"ayres, i; raseman, s; shih, a",low,med,high,high,high
2,"bager, s; mundaca, l",high,med,high,high,high
3,"carroll, j; lyons, s; denny, e",high,med,high,high,high
4,"houde, s; todd, a; sudarshan, a; flora, ja; ar...",low,med,high,high,high
5,"matsukawa, i.",high,med,high,high,high


,paper_id,criterion_1,criterion_2,criterion_5,criterion_6,criterion_7
0,"allcott, h",low,med,high,high,med
1,"ayres, i; raseman, s; shih, a",high,med,high,high,med
2,"bager, s; mundaca, l",high,med,high,high,high
3,"carroll, j; lyons, s; denny, e",low,med,high,high,med
4,"houde, s; todd, a; sudarshan, a; flora, ja; ar...",high,med,low,high,high
5,"matsukawa, i.",low,high,high,high,high


# Risk Level Per Criteria Comparison

In [17]:
def compare_rob_details(llm_df: pd.DataFrame, gt_df: pd.DataFrame,
                        titles: list, criteria_order: list) -> pd.DataFrame:
    """
    Compare LLM vs Ground Truth RoB details and return a clean DataFrame.

    The DataFrame contains:
      - All papers with title, criteria columns, overall, and row_accuracy.
      - A final row for column_accuracy.
      - The overall accuracy (based only on overall column) displayed in the last bottom right cell.

    Assumptions:
      - Titles and criteria_order are authoritative and overwrite dataset order/names.
      - Each criterion cell is formatted as "llm / gt".
    """

    title_col = PAPER_ID_COL

    def _select_and_standardize(df: pd.DataFrame) -> pd.DataFrame:
        candidate = [c for c in df.columns if c not in (title_col)]
        k = len(criteria_order)
        crit_cols = candidate[:k]
        view = df[[title_col] + crit_cols]
        rename_map = {old: new for old, new in zip(crit_cols, criteria_order)}
        view = view.rename(columns=rename_map)
        return view

    llm_idx = llm_df.set_index(title_col)
    gt_idx  = gt_df.set_index(title_col)

    llm_sel_raw = llm_idx.loc[titles].reset_index()
    gt_sel_raw  = gt_idx.loc[titles].reset_index()

    llm_sel = _select_and_standardize(llm_sel_raw)
    gt_sel  = _select_and_standardize(gt_sel_raw)

    def _norm(df: pd.DataFrame, cols: list) -> pd.DataFrame:
        out = df.copy()
        for c in cols:
            out[c] = out[c].astype(str).str.strip().str.lower()
        return out

    llm_norm = _norm(llm_sel, criteria_order)
    gt_norm  = _norm(gt_sel,  criteria_order)

    # Build comparison table
    comp = pd.DataFrame({"title": titles})
    for c in criteria_order:
        comp[c] = llm_norm[c] + " / " + gt_norm[c]

    # Accuracy calculations
    matches = pd.DataFrame({c: (llm_norm[c] == gt_norm[c]) for c in criteria_order})

    # Row-wise accuracy (per paper across criteria)
    row_acc = matches.mean(axis=1)
    
    # row-wise fraction
    row_acc_str = []
    total = len(criteria_order)
    for i, c in enumerate(titles):
        match = matches.sum(axis=1)[i]
        row_acc_str.append(f"{match}/{total} ({row_acc[i].round(3)})")
    
    comp["row_accuracy"] = row_acc_str

    # Column-wise accuracy (per criterion across all papers)
    col_acc = matches.mean(axis=0).round(3) # a list.
    
    # Column-wise fraction
    col_acc_str = []
    correct_answer = 0
    total = len(titles)
    for i, c in enumerate(criteria_order):
        match = matches.sum(axis=0)[i]
        col_acc_str.append(f"{match}/{total} ({col_acc[i]})")
        correct_answer += match

    # Add column_accuracy as the last row
    acc_row = {"title": "column_accuracy"}
    for i, c in enumerate(criteria_order):
        acc_row[c] = col_acc_str[i]
    acc_row["row_accuracy"] = ""

    comp = pd.concat([comp, pd.DataFrame([acc_row])], ignore_index=True)
    
    # Overall accuracy based only on overall column
    acc = (correct_answer / (len(titles)*len(criteria_order))).round(3)
    
    # Put overall accuracy at the bottom right cell.
    comp.iat[-1,-1] = f"overall: {acc}"

    return comp, acc

In [18]:
# Get column names from gt_risk_df.
excluded = [PAPER_ID_COL]
RISK_CRITERIA = [str(col) for col in gt_risk_df.columns if col not in excluded]
RISK_CRITERIA

['criterion_1', 'criterion_2', 'criterion_5', 'criterion_6', 'criterion_7']

In [19]:
# Generates risk-level comparisons per criteria.
cmp_risk_df, acc = compare_rob_details(llm_risk_df, gt_risk_df, PAPER_IDS, RISK_CRITERIA)

display(cmp_risk_df)
print(f"Overall accuracy: {acc}")

,title,criterion_1,criterion_2,criterion_5,criterion_6,criterion_7,row_accuracy
0,"allcott, h",low / low,med / med,high / high,high / high,high / med,4/5 (0.8)
1,"ayres, i; raseman, s; shih, a",low / high,med / med,high / high,high / high,high / med,3/5 (0.6)
2,"bager, s; mundaca, l",high / high,med / med,high / high,high / high,high / high,5/5 (1.0)
3,"carroll, j; lyons, s; denny, e",high / low,med / med,high / high,high / high,high / med,3/5 (0.6)
4,"houde, s; todd, a; sudarshan, a; flora, ja; ar...",low / high,med / med,high / low,high / high,high / high,3/5 (0.6)
5,"matsukawa, i.",high / low,med / high,high / high,high / high,high / high,3/5 (0.6)
6,column_accuracy,2/6 (0.333),5/6 (0.833),5/6 (0.833),6/6 (1.0),3/6 (0.5),overall: 0.7


Overall accuracy: 0.7


# Per Criteria Risk-Level Confusion Matrix + Metrics

In [20]:
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score

def evaluate_criteria(df_gt, df_llm, labels=["low", "med", "high"]):
    # Select only criteria columns.
    criteria_cols = [c for c in df_gt.columns if c.startswith("criterion")]
    
    # Flatten into 1-D arrays.
    y_true = df_gt[criteria_cols].values.ravel()
    y_pred = df_llm[criteria_cols].values.ravel()
    
    # Confusion matrix.
    cm = confusion_matrix(y_true, y_pred, labels=labels)
    cm_df = pd.DataFrame(cm,
                         index=[f"Actual {l}" for l in labels],
                         columns=[f"Pred {l}" for l in labels])
    
    # Per-class (per risk-level) metrics.
    precision, recall, f1, support = precision_recall_fscore_support(
        y_true, y_pred, labels=labels, zero_division=0
    )
    
    metrics_df = pd.DataFrame({
        "Class": labels,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Support": support
    })
    
    # Overall accuracy.
    acc = accuracy_score(y_true, y_pred)
    
    return cm_df, metrics_df, acc

In [21]:
cm_df, metrics_df, acc = evaluate_criteria(gt_risk_df, llm_risk_df)

print("Confusion Matrix:")
print(cm_df, "\n")

acc_df = pd.DataFrame([{
    "Class": "Overall Accuracy",
    "Precision": None,
    "Recall": None,
    "F1-score": None,
    "Support": None,
    "Accuracy": acc
}])

# Concatenate metrics with the accuracy row
combined_metrics_df = pd.concat([metrics_df, acc_df], ignore_index=True)
print(combined_metrics_df)

Confusion Matrix:
             Pred low  Pred med  Pred high
Actual low          1         0          3
Actual med          0         5          3
Actual high         2         1         15 

              Class  Precision    Recall  F1-score Support  Accuracy
0               low   0.333333  0.250000  0.285714       4       NaN
1               med   0.833333  0.625000  0.714286       8       NaN
2              high   0.714286  0.833333  0.769231      18       NaN
3  Overall Accuracy        NaN       NaN       NaN    None       0.7


## Save all results into an excel file.

In [22]:
import time
formatted_time = time.strftime("%Y-%m-%d_%H:%M:%S", time.localtime())

dfs = {
    'formatted_llm_results' : df_llm,
    'formatted_gt' : df_gt,
    'direct_comp' : direct_comparison_df,
    'per_paper_metrics' : per_paper_metrics_df,
    'per_criteria_metrics' : per_subcriteria_metrics_df,
    'risk_llm' : llm_risk_df,
    'risk_gt' : gt_risk_df,
    'risk_comp' : cmp_risk_df,
    'risk_confusion_matrix' : cm_df,
    'risk_metrics' : combined_metrics_df
}

with pd.ExcelWriter(f"output/results_{formatted_time}.xlsx", engine="openpyxl") as writer:
    for sheet_name, data in dfs.items():
        data.to_excel(writer, sheet_name=sheet_name, index=False)